Домашняя работа по вычислительной математике, задание 2.

Выполнил: Вязовцев Андрей Викторович, Б01-009

Вариант: о)

Результат (сначала запустить все ячейки ниже):

In [187]:
analit_solve = []
iter_solve = SolveRelax (my_A, my_f)

if (can_use_LU):
    analit_solve = SolveLU (my_A, my_f)
else:
    analit_solve = SolveGauss (my_A, my_f)

print ("Неувязка для аналитического решения: ", GetHitch (my_A, my_f, analit_solve))
print ("Неувязка для итерационного решения: ",  GetHitch (my_A, my_f, iter_solve))
print ("Число обусловленности матрицы mu = ", GetMu (my_A))
print ("lambda_max = ", GetLambdaMax (my_A), 
     "\nlambda_min = ", GetLambdaMin (my_A))

print ("\n\n\n")
print ("Аналитическое решение:\n", analit_solve)
print ("\nИтерационное решение:\n", iter_solve)

Неувязка для аналитического решения:  6.957936963747176e-14
Неувязка для итерационного решения:  8.689832925106965e-07
Число обусловленности матрицы mu =  1.4991943334502214
lambda_max =  11.999032564583974 
lambda_min =  8.000967435416026




Аналитическое решение:
 [0.08114061881585005, 0.16393716688650095, 0.24656644954998497, 0.329212627691554, 0.4118570992209515, 0.4945017431352375, 0.5771463696369266, 0.6597909978974621, 0.7424356259803365, 0.8250802540811565, 0.9077248821801637, 0.9903695102793542, 1.0730141383785259, 1.1556587664776996, 1.238303394576873, 1.3209480226760468, 1.4035926507752203, 1.4862372788743938, 1.5688819069735673, 1.6515265350727408, 1.7341711631719146, 1.8168157912710878, 1.8994604193702618, 1.982105047469435, 2.0647496755686086, 2.1473943036677827, 2.2300389317669556, 2.312683559866129, 2.395328187965303, 2.4779728160644763, 2.56061744416365, 2.6432620722628233, 2.725906700361997, 2.8085513284611707, 2.8911959565603444, 2.9738405846595177, 3.05648521275869

Необходимые библиотеки:

In [7]:
import numpy as np
from copy import deepcopy

Условие уравнения (матрица и свободные коэффициенты)

In [173]:
n = 100
a = 10.
b = 10.

def aij_ctor (x, y):
    if (x == y):
        return a;
    if (abs (y - x) == 1):
        return 1
    if (x - y == 2):
        return 1. / b
    return 0.

my_A = np.array ([[aij_ctor (x, y) for x in range (n)] for y in range (n)])
my_f = np.array ([i + 1 for i in range (n)])

# print (*A, sep = '\n')
# print (f)

Проверим, что все главные миноры больше 0, т.е. метод LU-разложения применим:

In [174]:
can_use_LU = True

for i in range (n):
    if (np.linalg.det ([A_j[0:(i+1)] for A_j in my_A[0:(i+1)]]) == 0):
        can_use_LU = False
        
if (can_use_LU):
    print ("Делай LU!") # мой случай
else:
    print ("LU-разложение невозможно")

Делай LU!


Считаем неувязку, число обусловленности $\mu$, $\lambda_{min}$, $\lambda_{max}$:

In [175]:
def Norm3Vec (x):
    return np.sqrt (np.sum (x * x))

def Norm3Matr (matr):
    w, v = np.linalg.eigh (matr)
    return np.max (np.abs (w))

def GetHitch (A, f, x):
    return Norm3Vec (np.matmul (A, x) - f)

def GetMu (A):
    return Norm3Matr (A) * Norm3Matr (np.linalg.inv (A))

def GetLambdaMax (matr):
    w, v = np.linalg.eigh (matr)
    return np.max (w)

def GetLambdaMin (matr):
    w, v = np.linalg.eigh (matr)
    return np.min (w)

Методы решения:

In [176]:
def SolveLU (A_const, f_const):
    lenA = len (A_const)
    A = deepcopy (A_const)
    f = deepcopy (f_const)
    
    L = [[0 for j in range (lenA)] for i in range (lenA)]
    U = deepcopy (L)
    for i in range (lenA):
        L[i][i] = 1
    
    get_Uij = lambda i, j:  A[i][j] - sum ([L[i][k] * U[k][j] for k in range (i)])
    get_Lij = lambda i, j: (A[i][j] - sum ([L[i][k] * U[k][j] for k in range (j)])) / U[j][j]
    
    for i in range (lenA):
        for j in range (i):
            L[i][j] = get_Lij (i, j)
        
        for j in range (i, lenA):
            U[i][j] = get_Uij (i, j)
            
    # solve LY = f   
    y = ReverseStrokeL (L, f)
    # solve Ux = y
    return ReverseStrokeU (U, y)

In [178]:
def SolveGauss (A_const, f_const):
    lenA = len (A_const)
    A = deepcopy (A_const)
    f = deepcopy (f_const)
    
    for i in range (lenA - 1):
        max_elem_ind = FindMaxElemInd (A, i)
        if (max_elem_ind != i):
            A[i], A[max_elem_ind] = deepcopy (A[max_elem_ind]), deepcopy (A[i])

        for j in range (i + 1, lenA):
            coef = A[j][i] / A[i][i]
            A[j] -= A[i] * coef
            f[j] -= f[i] * coef

    return ReverseStrokeU (A, f)
        
        
def FindMaxElemInd (A, i_row):
    max_elem_ind = i_row
    for j in range (i_row + 1, len (A)):
        if (abs (A[j][i_row]) > abs (A[max_elem_ind][i_row])):
            max_elem_ind = j
    return max_elem_ind

In [179]:
# a11 a12 a13   x1   f1
#  0  a22 a23 X x2 = f2
#  0   0  a33   x3   f3
def ReverseStrokeU (U, f):
    lenU = len (U)
    x = [0. for i in range (lenU)]
    
    for i in range (lenU - 1, -1, -1):
        x_i = 0.
        
        for j in range (i + 1, lenU):
            x_i += U[i][j] * x[j]
        x[i] = (f[i] - x_i) / U[i][i]
    
    return x

# a11  0   0    x1   f1
# a21 a22  0  X x2 = f2
# a31 a32 a33   x3   f3
def ReverseStrokeL (L, f):
    x = ReverseStrokeU (np.flip (L, (0, 1)), np.flip (f))
    return np.flip (x)

In [184]:
def SolveRelax (A, f):
    lenA = len (A)
    w    = 1.726
    x_0  = np.linspace (0, 2, lenA) # random
    EPS  = 1e-6
    
    L = np.array ([[0. for j in range (lenA)] for i in range (lenA)])
    D = deepcopy (L)
    U = deepcopy (L)
    
    for i in range (lenA):
        for j in range (lenA):
            if (j > i):
                U[i][j] = A[i][j]
            elif (j < i):
                L[i][j] = A[i][j]
            elif (j == i):
                D[i][j] = A[i][j]
    
    for i in range (lenA):
        for j in range (lenA):
            test = np.abs (U[i][j] + L[i][j] + D[i][j] - A[i][j])
            if (test != 0):
                print (i, j, U[i][j], L[i][j], D[i][j], A[i][j])
                
    D_wL_pow_m1 = np.linalg.inv (D + w * L)
    wm1_D_wU    = (w - 1) * D + w * U
    
    coef_before_x = -np.matmul (D_wL_pow_m1, wm1_D_wU)
    coef_free     = w * np.matmul (D_wL_pow_m1, f)
    
    do_iter = lambda x: np.matmul (coef_before_x, x) + coef_free

    x = x_0
    while (np.abs (GetHitch (A, f, x)) > EPS):
        x = do_iter (x)
        
    return x